In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
# import spacy
# nlp = spacy.load('en_core_web_lg')
# def tokenizer(corpus):
#     return [str(token) for token in nlp(corpus)]
# a = '"River walk that led me to my secret family: after being adopted as a child, Katharine Norbury reveals the emotional journey to reconnect with her biological mother"'.replace('"', '')
# tokenizer(a)

## Load and prepare data

In [3]:
pretrained_file = './pretrained_embedding/glove.840B.300d.txt'
train_file = 'data/train.csv'
test_file = 'data/test.csv'
max_size = 128
min_freq = 2
batch_size = 64

In [4]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, max_size, min_freq, batch_size)

In [5]:
AttractiveData.LABEL.vocab.freqs

Counter({'4.0': 226,
         '2.3333333333333335': 194,
         '4.5': 43,
         '3.3333333333333335': 313,
         '3.6666666666666665': 260,
         '2.6666666666666665': 281,
         '2.0': 135,
         '2.5': 36,
         '1.6666666666666667': 28,
         '3.0': 354,
         '4.333333333333333': 82,
         '4.666666666666667': 29,
         '1.5': 16,
         '3.5': 22,
         '1.3333333333333333': 4,
         '1.0': 5,
         '5.0': 12})

In [6]:
batch = next(iter(AttractiveData.trainloader))
batch


[torchtext.data.batch.Batch of size 64]
	[.Headline]:[torch.cuda.LongTensor of size 64x64 (GPU 0)]
	[.Label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

## Start to train

In [7]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_name = './model/AttractiveNet'
num_workers = 10
input_dim = len(AttractiveData.TEXT.vocab)
embedding_dim = 300
hidden_dim = 128
output_dim = 1
log_steps = 10
epochs = 30
lr = 1e-2
num_layers = 1
dropout = 0.1
pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([4317, 300])


In [8]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [9]:
AttractiveTrainer = AttractiveTrainer(save_name, log_steps, epochs, lr, timestr, AttractiveData.device, AttractiveData.trainloader, AttractiveData.testloader, input_dim, embedding_dim, hidden_dim, output_dim, pretrained_embeddings, dropout, num_layers)

In [10]:
AttractiveTrainer.model

AttractiveNet(
  (embedding): Embedding(4317, 300)
  (rnn): RNN(300, 128, num_layers=4, dropout=0.1)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [11]:
AttractiveTrainer.train()

EP:0 | lr: 0.01: 100%|| 32/32 [00:00<00:00, 69.02it/s]
EP: train | lr: 0.01: 100%|| 32/32 [00:00<00:00, 217.26it/s]
EP:1 | lr: 0.01:  28%|| 9/32 [00:00<00:00, 88.25it/s]
EP_train | avg_loss: 9.68303108215332 |
EP:1 | lr: 0.01: 100%|| 32/32 [00:00<00:00, 86.92it/s]
EP: train | lr: 0.01: 100%|| 32/32 [00:00<00:00, 219.65it/s]
EP:2 | lr: 0.01:  28%|| 9/32 [00:00<00:00, 80.77it/s]
EP_train | avg_loss: 9.658508211374283 |
EP:2 | lr: 0.01: 100%|| 32/32 [00:00<00:00, 84.75it/s]
EP: train | lr: 0.01: 100%|| 32/32 [00:00<00:00, 219.56it/s]
EP:3 | lr: 0.01:  28%|| 9/32 [00:00<00:00, 81.51it/s]
EP_train | avg_loss: 9.737947136163712 |
EP:3 | lr: 0.01: 100%|| 32/32 [00:00<00:00, 81.17it/s]
EP: train | lr: 0.01: 100%|| 32/32 [00:00<00:00, 203.67it/s]
EP:4 | lr: 0.01:  28%|| 9/32 [00:00<00:00, 83.63it/s]
EP_train | avg_loss: 9.681198596954346 |
EP:4 | lr: 0.01: 100%|| 32/32 [00:00<00:00, 88.27it/s]
EP: train | lr: 0.01: 100%|| 32/32 [00:00<00:00, 195.67it/s]
EP:5 | lr: 0.01:  28%|| 9/32 [00:00<00:00

In [12]:
def predict_attractive(sentence):
    tokens = AttractiveData.tokenizer(sentence)
    indexed = [AttractiveData.TEXT.vocab.stoi[t] for t in tokens]
    tensor = torch.LongTensor(indexed).to(AttractiveData.device)

    tensor = tensor.unsqueeze(1)
    prediction = AttractiveTrainer.model(tensor)
    
    return prediction[0][0][0].item()

In [13]:
a = "Traditional Bombay-style Cafe beats Heston Blumenthal's Michelin-starred restaurant to be crowned the best in the Uk"
prediction = predict_attractive(a)
prediction

0.5558767914772034

In [12]:
pred, true = AttractiveTrainer.evaluate(AttractiveTrainer.test_loader, 'test')

EP: test | lr: 0.01:   0%|| 0/4 [00:00<?, ?it/s]


AttributeError: 'Example' object has no attribute 'Text'

## Below is testing

In [10]:
a = df_test['ID'].to_list()

In [17]:
from sklearn.metrics import mean_squared_error

In [9]:
mean_squared_error(a, b)
# Training all 3.0 got mse = 0.5545

0.5545343137254902

In [20]:
df_test['Label'] = b

In [21]:
df_test[['ID', 'Label']].to_csv('all_3.csv', index=False)